<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0;margin-top: 0;">
        <img src="../pics/NationalDataPlatform_logo.png" alt="WiFire Logo" style="width: 179px; margin-bottom: 0px;">
    </div>    
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 0;">
        <img src="../pics/logo_UCSD.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="../pics/sdsclogo-plusname-horiz-red.jpg" alt="San Diego Supercomputer Center Logo" width="300"/>
    </div>
</div>
<h1 style="text-align: center; font-size: 24px; margin-top: 0;">NSF National Data Platform (NDP)</h1>
<h3 style="text-align: center; font-size: 18px; margin-top: 10px;">LLM as a Service Tutorial</h3>
<div style="margin: 20px 0;">
    <p align="justify"> Large Language Models are a powerful AI tool with multiple applications in both research and education, given their capacity to process big amounts of information to generate human-like language.</p>
    <p align="justify"> Understanding today's relevance of LLM's, the National Data Platform (NDP) has developed an LLM service to contribute to the research and education goals of its users.</p>
    <p align="justify"> In this guide, we are covering the use of an LLM as an NDP service. The main purpose of this demo is to showcase how this service works by submitting a series of sample queries, as well as comparing the performance when adding new documentation to a model. The main goal is to allow the user to identify the potential use cases of this service.</p>
</div>

<center>
    <div style="text-align: right; padding: 5px;">
        <p style="text-align: right;"><strong>Contact:</strong><a href="https://docs.google.com/forms/d/e/1FAIpQLSfzjlc0Sw2fTFTKArOZ0ffKNdVcPivf218kLXkBKfobGPbDMw/viewform"> NDP Issue Reporting Form </a></p>
    </div>
</cente


<div style="display: flex; align-items: center; justify-content: flex-start; margin-top: 20px; border-top: 1px solid #ccc; padding-top: 20px;">
    <img src="https://new.nsf.gov/themes/custom/nsf_theme/components/images/logo/logo-desktop.svg" alt="NSF Logo" style="width: 120px; margin-right: 10px;">
    <p style="font-size: 12px;">The National Data Platform was funded by NSF 2333609 under CI, CISE Research Resources programs. Any opinions, findings, conclusions, or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the funders.</p>
</div>
<hr>

### What is an LLM?

An LLM, or Large Language Model, is a type of artificial intelligence designed to understand and generate human-like text based on the data it's been trained on. By adding a vast amount of text from different sources and context's (web, books, papers, among others) LLM's are capable to identify the patterns of the human language under different contexts and provide responses to complex questions. 

LLM's posses a huge potential in both research and education, given their capability to quickly process and summarize a vast amount of information. LLM's can bee seen as a powerful tool to accelerate learning, facilitate the sharing of knowledge, process and generate big amounts of data, generate new hypothesis questions, among other uses.

### ClimateGPT

For this demo, we are using the [ClimateGPT](https://climategpt.ai/) model. Climate GPT was developed by team of researchers at RWTH Aachen University, in collaboration with Erasmus AI and others, for interdisciplinary research focused on climate change. They constructed 7B models from the ground up, utilizing a scientifically-oriented dataset of 300 billion tokens.  

The model was made publicly available through [HuggingFace](https://huggingface.co/eci-io/climategpt-70b) in January 2024. It comes in well-maintained 7B, 13B, and 70B versions, built upon the Llama 2 architecture and leveraging a dataset of 4.2 billion tokens.

### Hands on

We will start using LLM as a service. First, we will make some questions to the vanilla ClimateGPT to see its overall performance. Then, we will extend the chorpus of the model by adding a new document. We expect the model to be able to answer questions based on the new document.

### A. Set Up

We start by importing relevant libraries and functions. Also, we call the model and connect to the server's API. 

In [ ]:
import requests
import os
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.document_loaders import PyPDFLoader

MODEL = 'eci-io/climategpt-7b' # We specify the model tu use
os.environ['OPENAI_API_BASE'] = 'http://localhost:8000/v1'
os.environ['OPENAI_API_KEY'] = 'EMPTY'

### Test that LLM API is up

In [ ]:
!curl  -X 'GET' 'http://localhost:8000/v1/models' -H 'accept: application/json'

### The conversation code

In the following cells, we define a series of functions to interact with ClimateGPT in a form of a conversation.

In [ ]:
def save_answer(question, answer):
    with open('output.txt', 'a') as file:
        file.write(question+'\n')
        file.write(answer+'\n\n')
    

def query(question, chorpus):
    url = "http://localhost:8000/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
    }
    
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": question}],
    }
    if chorpus:
        answer = index.query(question, llm=llm)
    else:
        response = requests.post(url, json=data, headers=headers)
        answer = response.json()['choices'][0]['message']['content']
    
    print(f"\nAnswer: {answer}")

    return question, answer


def conversation(corpus=False):
    answer = None
    question = None
    while True:
        user_input = input("Question (q=quit, s=save previous answer): ")
        if user_input=='q':
            break
        if user_input=="s" and answer:
            save_answer(question, answer)
            answer = None
        else:
            question, answer = query(user_input, corpus)

### B. Running the conversation

Now we will start a conversation with ClimateGPT. Some sample questions to begin:

- What AI/ML methods are used for wildfire data analysis?
- Summarize the impact of climate change on wildfires. Include the nature of damages caused by climate-related changes.
- What are the biodiversity impacts of wildfire?

After each question, you can:
- Type a new question
- Type "s" if you want to save your previous question
- Type "q" to quit the chat

In [ ]:
conversation()

### C. Adding new context to ClimateGPT

Now we will extend the capacity of the model by adding new documentation. We will use the following document:

- ["Strengthening and Democratizing the U.S. Artificial Intelligence Innovation Ecosystem: An Implementation Plan for a National Artificial Intelligence Research Resource" (NAIRR-TF-Final-Report-2023.pdf)](https://www.ai.gov/wp-content/uploads/2023/01/NAIRR-TF-Final-Report-2023.pdf)

In the following cell, we will load and embed our document, and index it for retrieval from our Vector Store.

This cell might take a few minutes.

In [ ]:
# We load the new document to the model
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
file = '../PDF Documents/NAIRR-TF-Final-Report-2023.pdf'
loader = PyPDFLoader(file)
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
llm = ChatOpenAI(model=MODEL)

Once we have loaded our document, we can start making questions. Some questions that we can use to compare results are the following:

* What task forces are recommended in the report?  
* List the facilities that a NAIRR resource should provide with respect to (a) computing, (b) AI/ML models, and (c) data.  
* What committees does the report refer to?  
* What are NAIRR's goals with respect to human capital?

Let's start with the first question: *What task forces are recommended in the report?* 

1. Run the *PRE-Context* cell and ask the question. You will notice that the model will respond to not possess any knowledge of a report.
2. Type q to close the conversation
3. Go to the next cell *POST*, and repeat the process. Now the model will provide an answer relying on the document.

#### PRE-Context

In [ ]:
# PRE - In this cell, we will ask the question without giving any context to ClimateGPT
conversation()

Looking at the responses, we can confirm the model is given proper answers which are constructed taking the added document as context.

#### POST-Context

In [ ]:
# POST - We add True to indicate the model to make use of the new document
conversation(True)